In [ ]:
!pip install pygame
import sys, os
import pygame
import numpy as np

     |████████████████████████████████| 11.8MB 6.4MB/s 
pygame 2.0.1 (SDL 2.0.14, Python 3.7.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
HIT_BRIGHTNESSES = np.arange(1, 0, -.05)
HIT_DURATION = 600
HIT_GAP = HIT_DURATION / len(HIT_BRIGHTNESSES)

def hit(start_time, current_time):
    diff = current_time-start_time
    if diff >= HIT_DURATION:
        return (0, 0, 0)
    val = int(HIT_BRIGHTNESSES[int(diff/HIT_GAP)] * 255)
    return (val, val, val)

HIT_DURATION_BYTE=100
def hit_byte(start_time, current_time):
    diff = current_time-start_time
    if diff >= HIT_DURATION_BYTE:
        return 0
    return 1

BEAT_THRESH = 100
def beat_detector(diff, beat_length):
    if (diff - int(diff/beat_length) * beat_length) < BEAT_THRESH:
        return True
    else:
        return False
    
def hit_normalize(start_time, current_time):
    diff = current_time - start_time
    if diff >= HIT_DURATION:
        return 0
    return min(1 - (float(diff) / HIT_DURATION), 1)
    
# state = {beat:<0/1>, hit:<0-1>}
BEAT_RECT = pygame.Rect(30, 30, 640, 30)
HIT_RECTS = [(pygame.Rect(90 * i + 30, 100, 30, 30), (int(255 * (1-i/10)), int(255 * i/10), 255)) for i in range(10)]
DIFF_MULTIPLIERS = [1, .8, .3]
run_once = True
def draw(screen, state):
    global run_once
    if state['beat'] == 1:
        beat_color = (255, 255, 255)
    else:
        beat_color = (0, 0, 0)
    pygame.draw.rect(screen, beat_color, pygame.Rect(30, 30, 640, 30))

    #draw animation
    if state['hit'] == 0:
        for hit_rect in HIT_RECTS:
            pygame.draw.rect(screen, (0, 0, 0), hit_rect[0])
        return;
    
    index = int((1 - state['hit']) * 10)
#     for hit_rect in HIT_RECTS:
#         pygame.draw.rect(screen, [int(i * state['hit']) for i in hit_rect[1]], hit_rect[0])
#         pygame.draw.rect(screen, (0, 0, 0), hit_rect[0])
    for i, hit_rect in enumerate(HIT_RECTS):
        diff = np.abs(index - i)
        if diff < len(DIFF_MULTIPLIERS):
            multiplier = DIFF_MULTIPLIERS[diff]
        else:
            multiplier = 0
        pygame.draw.rect(screen, [int(i * multiplier) for i in hit_rect[1]], hit_rect[0])

    pygame.draw.rect(screen, HIT_RECTS[index][1], HIT_RECTS[index][0])

# max for NUM_LEDS is 256/4 = 64
NUM_LEDS = 18
def encode_byte(state, new_hit, slide):
    index = int((1-state['hit']) * NUM_LEDS) #index=10 means all dark
    beat = state['beat']
    if slide:
        return 3;
    return new_hit;
    return index * 4 + new_hit * 2 + beat
    
def slider_duration(length, multiplier, beat_length):
    return length / (multiplier * 100) * beat_length

In [ ]:
hit_objects = beatmap.hit_objects.copy();
timing_objects = beatmap.timing_objects.copy();

hit_times2 = None
if beatmap2 is not None:
    hit_times2 = [hit_object.time for hit_object in beatmap2.hit_objects]
else:
    hit_times2 = hit_times.copy()
print(len(hit_times2))
print(len(hit_objects))

# music_file = folder + beatmap.general_settings.audio_file_name[1:]
pygame.init()
infoObject = pygame.display.Info()


screen_w = 700
screen_h = 700

# Set up the drawing window
screen = pygame.display.set_mode([screen_w, screen_h])
pygame.mixer.music.load(music_file)

new_hit = 0

start_hit = -1000
interval_time = -1000
INTERVAL_THRESH = 20


timing_object = None
timing_object = timing_objects[0]
slider_multiplier = beatmap.difficult_settings.slider_multiplier
slider_multiplier_multiplier = 1;
slide_end = -1;

state = {'beat': 0, 'hit': 0}

pygame.mixer.music.play(0)
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            print('EXIT DETECTED')
            running = False;    
    if pygame.mixer.music.get_busy() == False:
        print('EXIT BC MUSIC STOP')
        running = False;
    else:
        music_time = pygame.mixer.music.get_pos()
    
    if len(timing_objects) > 0 and music_time > timing_objects[0].time-50:
        temp = timing_objects.pop(0)
        if temp.uninherited == 1:
            timing_object = temp
            slider_multiplier_multiplier = 1
        else:
            slider_multiplier_multiplier = 100/(-temp.beat_length)
    
    if timing_object is not None and beat_detector(music_time-timing_object.time, timing_object.beat_length):
        state['beat'] = 1
    else:
        state['beat'] = 0

    # Fill the background with white
        
    if(len(hit_times2) != 0 and music_time > hit_times2[0]-10):
        #HIT
        hit_times2.pop(0)
        start_hit = music_time
    if(len(hit_objects) != 0 and music_time > hit_objects[0].time-10):
        hit_obj = hit_objects.pop(0)
        if HitObjectType.SLIDER in hit_obj.type:
            slide_end = music_time + slider_duration(float(hit_obj.length), slider_multiplier_multiplier * slider_multiplier , timing_object.beat_length)
        elif HitObjectType.CIRCLE in hit_obj.type:
            new_hit = 1;
    state['hit'] = hit_normalize(start_hit, music_time)
#     screen.fill(hit(start_hit, music_time))
    
    if(music_time - interval_time > INTERVAL_THRESH):
#         ser.write(bytes([hit_byte(start_hit, music_time)]))

#         ser.write(bytes([encode_byte(state, new_hit, slide_end > music_time)]))
        new_hit = 0
        interval_time = music_time
    
    draw(screen, state)
    # Flip the display
    pygame.display.flip()

# Done! Time to quit.
state['beat'] = 0
state['hit'] = 0
# ser.write(bytes([encode_byte(state, 0, 0)]))
pygame.quit()